In [1]:
import os
os.chdir("..")
print(os.getcwd())


C:\Users\eac84\Universiteit Antwerpen\Spark Automated Analysis - General\OrBITS


In [2]:
%load_ext autoreload
%load_ext line_profiler

In [3]:
%matplotlib qt
%autoreload 2

In [4]:
from retna.networks import *
from retna.training import *
from retna.data_loader import *
from retna.view import *
from retna.prune import *
from retna.hook_tools import *
from retna.simulate_data import *

In [5]:
import torch
import glob as glob
import matplotlib.pyplot as plt

## Load Model

In [54]:
h_chans = [10,10,8,8,4,4]
model = Retna_V1(1,6, h_chans)

In [55]:
SimLoader = SimDataset(5)

In [23]:
SimLoader

In [56]:
train_model_x( model, SimLoader, num_epochs=50 )

Start Loss: 0.9229706525802612
Epoch 0/49	..........Loss: 0.681741	Caching improved model	
Epoch 1/49	..........Loss: 0.623851	Caching improved model	
Epoch 2/49	..........Loss: 0.611456	Caching improved model	
Epoch 3/49	..........Loss: 0.539756	Caching improved model	
Epoch 4/49	..........Loss: 0.515989	Caching improved model	
Epoch 5/49	..........Loss: 0.519480	
Epoch 6/49	..........Loss: 0.501731	Caching improved model	
Epoch 7/49	..........Loss: 0.478159	Caching improved model	
Epoch 8/49	..........Loss: 0.497533	
Epoch 9/49	..........Loss: 0.535287	
Epoch 10/49	..........Loss: 0.497666	lr: 0.0005	
Epoch 11/49	..........Loss: 0.494479	
Epoch 12/49	..........Loss: 0.476390	Caching improved model	
Epoch 13/49	..........Loss: 0.477980	
Epoch 14/49	..........Loss: 0.468906	Caching improved model	
Epoch 15/49	..........Loss: 0.468725	Caching improved model	
Epoch 16/49	..........Loss: 0.464299	Caching improved model	
Epoch 17/49	..........Loss: 0.474959	
Epoch 18/49	..........Loss: 0.4

Retna_V1(
  (blocks): ModuleList(
    (0): Sequential(
      (0): Conv2d(1, 11, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): Conv2d(11, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(11, 21, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): Conv2d(21, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(11, 19, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): Conv2d(19, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (3): Sequential(
      (0): Conv2d(9, 17, kernel_size=(3, 3), stride=(1, 1), padding=

In [13]:
print_comparisons_x(model,SimLoader)

AttributeError: 

In [57]:
print_mosaic_x(SimLoader,model,  n=5)

tensor(0.3654, grad_fn=<MeanBackward0>)
tensor(0.3625, grad_fn=<MeanBackward0>)
tensor(0.3928, grad_fn=<MeanBackward0>)
tensor(0.3476, grad_fn=<MeanBackward0>)
tensor(0.3658, grad_fn=<MeanBackward0>)
tensor(0.3342, grad_fn=<MeanBackward0>)
tensor(0.3443, grad_fn=<MeanBackward0>)
tensor(0.3293, grad_fn=<MeanBackward0>)
tensor(0.4199, grad_fn=<MeanBackward0>)
tensor(0.3218, grad_fn=<MeanBackward0>)
tensor(0.4006, grad_fn=<MeanBackward0>)
tensor(0.3568, grad_fn=<MeanBackward0>)
tensor(0.3578, grad_fn=<MeanBackward0>)
tensor(0.3504, grad_fn=<MeanBackward0>)
tensor(0.3813, grad_fn=<MeanBackward0>)
tensor(0.3348, grad_fn=<MeanBackward0>)
tensor(0.3563, grad_fn=<MeanBackward0>)
tensor(0.3271, grad_fn=<MeanBackward0>)
tensor(0.4118, grad_fn=<MeanBackward0>)
tensor(0.3453, grad_fn=<MeanBackward0>)
tensor(0.3869, grad_fn=<MeanBackward0>)
tensor(0.4060, grad_fn=<MeanBackward0>)
tensor(0.3378, grad_fn=<MeanBackward0>)
tensor(0.3727, grad_fn=<MeanBackward0>)
tensor(0.3303, grad_fn=<MeanBackward0>)


## Load Simulated Data into H5 Format

In [24]:
image, mask = generate_random_data(120, 120, count=500)
#image = image.transpose(0,3,1,2)[:,0:1]

In [58]:
plt.figure()
plt.imshow(image[0,0])
plt.figure()
plt.imshow(mask[0,2])

In [33]:
root = ""  ## Add desination folder here
filename = root + "example_data.h5"

In [34]:
import h5py 
with h5py.File(filename, 'w') as fh:
    dset = fh.create_dataset("input", data=image , compression="lzf" )
    dset = fh.create_dataset("target", data=mask , compression="lzf" )

## Load Data from H5 Format

In [39]:
data = H5_Handler([filename], datasets=["input","target"])
Loader = DataLoader(data)

In [47]:
## Specify what header the target data is in. 
Loader.dataset.dataset_names[1]= "target"  
Loader.dataset.channels[1] = None

In [ ]:
## Specify what header the target data is in. 

In [41]:
model = model.to("cpu")
Loader.dataset.device = "cpu"

In [60]:
train_model( model, Loader, num_epochs=10 )

Start Loss: 0.5858020010173962
Epoch 0/9	....................Loss: 0.584026	Caching improved model	
Epoch 1/9	....................Loss: 0.580010	Caching improved model	
Epoch 2/9	....................Loss: 0.574997	Caching improved model	
Epoch 3/9	....................Loss: 0.570901	Caching improved model	
Epoch 4/9	....................Loss: 0.564390	Caching improved model	
Epoch 5/9	....................Loss: 0.559503	Caching improved model	
Epoch 6/9	....................Loss: 0.555679	Caching improved model	
Epoch 7/9	....................Loss: 0.549265	Caching improved model	
Epoch 8/9	....................Loss: 0.544060	Caching improved model	
Epoch 9/9	....................Loss: 0.538227	Caching improved model	
Best val loss: 0.538227


Retna_V1(
  (blocks): ModuleList(
    (0): Sequential(
      (0): Conv2d(1, 11, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): Conv2d(11, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(11, 21, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): Conv2d(21, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(11, 19, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): Conv2d(19, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (3): Sequential(
      (0): Conv2d(9, 17, kernel_size=(3, 3), stride=(1, 1), padding=

In [45]:
print_comparisons(model,Loader)

AttributeError: 'Retna_V1' object has no attribute 'dataset'

In [61]:
print_mosaic(Loader,model,m=5,n=10)

[58]
tensor(0.2943, grad_fn=<MeanBackward0>)
[194]
tensor(0.3707, grad_fn=<MeanBackward0>)
[272]
tensor(0.2831, grad_fn=<MeanBackward0>)
[275]
tensor(0.2986, grad_fn=<MeanBackward0>)
[304]
tensor(0.2177, grad_fn=<MeanBackward0>)
[4]
tensor(0.3129, grad_fn=<MeanBackward0>)
[201]
tensor(0.3117, grad_fn=<MeanBackward0>)
[284]
tensor(0.3645, grad_fn=<MeanBackward0>)
[151]
tensor(0.3552, grad_fn=<MeanBackward0>)
[131]
tensor(0.2210, grad_fn=<MeanBackward0>)
[292]
tensor(0.2145, grad_fn=<MeanBackward0>)
[412]
tensor(0.2665, grad_fn=<MeanBackward0>)
[144]
tensor(0.2094, grad_fn=<MeanBackward0>)
[105]
tensor(0.3752, grad_fn=<MeanBackward0>)
[219]
tensor(0.2837, grad_fn=<MeanBackward0>)
[274]
tensor(0.2813, grad_fn=<MeanBackward0>)
[451]
tensor(0.3743, grad_fn=<MeanBackward0>)
[273]
tensor(0.2746, grad_fn=<MeanBackward0>)
[303]
tensor(0.2271, grad_fn=<MeanBackward0>)
[128]
tensor(0.3010, grad_fn=<MeanBackward0>)
[234]
tensor(0.3295, grad_fn=<MeanBackward0>)
[321]
tensor(0.2421, grad_fn=<MeanBac

## Load Data from Tiff Format

In [ ]:
#wip